In [28]:
from lxml import etree
# import requests

tree =etree.parse("src/web_page.html")
print(tree)

In [ ]:
title_element = tree.find("body/ul/li")
print(title_element.text)


Data scraping course.
GeekBrain's link: Developer


In [5]:
list_item = tree.findall("body/ul/li")
print(list_item)
for li in list_item: 
    a = li.find("a")
    if a is not None:
        print(f'{li.text.strip()} {a.text}')
    else:
        print(li.text)

[<Element li at 0x1ee896530c0>, <Element li at 0x1ee89653800>]
Data scraping course.
GeekBrain's link: Developer


In [6]:
title_element = tree.xpath("//title")
print(title_element[0].text)
title_element = tree.xpath("//p/text()")[0]
print(title_element)
list_item = tree.findall("body/ul/li")

This is title of the page
Hello GeekBrains


In [10]:
list_item = tree.xpath("//li")
for li in list_item:
    print(etree.tostring(li))


b'<li id="myID">Data scraping course.</li>\n'
b'<li class="myClass">GeekBrain\'s link:\n<a href="https://gb.ru/geek_university/developer">Developer</a>\n</li>\n'


In [13]:
list_item = tree.xpath("//li")
for li in list_item:
    text = tree.xpath('//text()')
    print(text)


['\n', '\n', 'This is title of the page', '\n', '\n', '\n', 'Hello GeekBrains', '\n', '\n', 'Data scraping course.', '\n', "GeekBrain's link:\n", 'Developer', '\n', '\n', '\n', '\n']
['\n', '\n', 'This is title of the page', '\n', '\n', '\n', 'Hello GeekBrains', '\n', '\n', 'Data scraping course.', '\n', "GeekBrain's link:\n", 'Developer', '\n', '\n', '\n', '\n']


In [26]:
list_item = tree.xpath("//li")
for li in list_item:
    text = map(str.strip, li.xpath("//text()"))
    print(list(text))

['', '', 'This is title of the page', '', '', '', 'Hello GeekBrains', '', '', 'Data scraping course.', '', "GeekBrain's link:", 'Developer', '', '', '', '']
['', '', 'This is title of the page', '', '', '', 'Hello GeekBrains', '', '', 'Data scraping course.', '', "GeekBrain's link:", 'Developer', '', '', '', '']


In [36]:
list_item = tree.xpath("//ul/descendant::li")
for li in list_item:
    text = ''.join(map(str.strip, li.xpath("//text()")))
    print(text)

This is title of the pageHello GeekBrainsData scraping course.GeekBrain's link:Developer
This is title of the pageHello GeekBrainsData scraping course.GeekBrain's link:Developer


In [38]:
html = tree.getroot()
title_element = html.cssselect("title")
print(title_element[0].text)

This is title of the page


In [42]:
html = tree.getroot()
title_element = html.cssselect("li")
print(title_element[0].text)

Data scraping course.


In [ ]:

import requests
from fake_useragent import UserAgent
from lxml import html

ua = UserAgent()
url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
headers = {'User-Agent': ua.random}
session = requests.Session()
response = session.get(url, headers=headers)
tree = html.fromstring(html=response.content)
movies = tree.xpath('.//div[contains(@class, "cli-title")]/a/h3/text()')
# print(response.status_code)


# try:
    m = {
    'name': movie.xpath(".//div[contains(@class, 'cli-title')]/a/h3/text()")[0],
    'release_year': movie.xpath(".//div[contains(@class, 'cli-title-metadata')]/span/text()")[0],
    'position': movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/text()")[0],
    'titlemeter': movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/span/svg[contains(@class, 'sc-db6887cf-0')]/@class")[0],
    'position_change': movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/span/text()")[0]
    }
#     all_movies.append(m)
# except IndexError:
# # Обрабатываем случаи, когда какой-то элемент не был найден
#     continue

#     pprint(all_movies)

In [56]:
import requests
from fake_useragent import UserAgent
from lxml import html

ua = UserAgent()
url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
headers = {'User-Agent': ua.random}
session = requests.Session()
response = session.get(url, headers=headers)
tree = html.fromstring(html=response.content)
movies = tree.xpath(".//div[contains(@class, 'cli-title')]")
all_movies = []
for movie in movies:
    m = {
    'name': movie.xpath(".//div[contains(@class, 'cli-title')]/a/h3/text()")[0],
    'release_year': movie.xpath(".//div[contains(@class, 'cli-title-metadata')]/span/text()")[0],
    'position': movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/text()")[0],
    'titlemeter': movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/span/svg[contains(@class, 'sc-db6887cf-0')]/@class")[0],
    'position_change': movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/span/text()")[0]
    }
    all_movies.append(m)
print(all_movies)
print(len(all_movies))


IndexError: list index out of range

In [ ]:
import requests
from fake_useragent import UserAgent
from lxml import html
from pymongo import MongoClient
ua = UserAgent()
url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
headers = {'User-Agent': ua.random}
session = requests.Session()



# URL страницы IMDb
# url = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"

# Отправка GET-запроса
# response = requests.get(url)
response = session.get(url, headers=headers)
response.raise_for_status()  # Проверка на ошибки
print(response.status_code)
# Парсинг HTML
tree = html.fromstring(response.content)

# Извлечение данных о фильмах
movies = tree.xpath('//ul[contains(@class,"ipc-metadata-list")]/li')

# Подключение к MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['imdb_database']  # Название базы данных
collection = db['movies']  # Название коллекции

# Сохранение данных в MongoDB
for movie in movies:
    title = movie.xpath('/div//h3[@class="ipc-title__text"]')[0]
    # year = movie.xpath('.//td[@class="titleColumn"]/span/text()')[0]
    rating = movie.xpath('import requests
from fake_useragent import UserAgent
from lxml import html
from pymongo import MongoClient

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def send_get_request(url, headers):
    session = requests.Session()
    response = session.get(url, headers=headers)
    response.raise_for_status()
    return response

def parse_html(response):
    tree = html.fromstring(response.content)
    return tree

def extract_movie_data(tree):
    movies = tree.xpath('//ul[contains(@class,"ipc-metadata-list")]/li')
    return movies

def connect_to_mongodb():
    client = MongoClient('mongodb://localhost:27017/')
    db = client['imdb_database']
    collection = db['movies']
    return collection

def save_movie_data(collection, movie):
    title = movie.xpath('/div//h3[@class="ipc-title__text"]')[0]
    rating = movie.xpath('/div//span[@class="ipc-rating-star--rating"]')[0]
    movie_data = {
        'title': title.text,
        'rating': rating.text
    }
    collection.insert_one(movie_data)

def main():
    url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
    headers = {'User-Agent': get_random_user_agent()}
    response = send_get_request(url, headers)
    tree = parse_html(response)
    movies = extract_movie_data(tree)
    collection = connect_to_mongodb()
    for movie in movies:
        save_movie_data(collection, movie)
    print("Данные успешно сохранены в MongoDB.")

if __name__ == "__main__":
    main()/div//span[@class="ipc-rating-star--rating"]')[0]
    print(title.text)
    print(rating.text)
    # Создание документа
    movie_data = {
        'title': title,
        # 'year': year,
        # 'rating': rating
    }
    # print(movie_data)
    # Вставка в коллекцию
    # collection.insert_one(movie_data)

print("Данные успешно сохранены в MongoDB.")


200


IndexError: list index out of range

In [64]:
import requests
from fake_useragent import UserAgent
from lxml import html
from pymongo import MongoClient
ua = UserAgent()
url = 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'
headers = {'User-Agent': ua.random}
session = requests.Session()



# URL страницы IMDb
# url = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"

# Отправка GET-запроса
# response = requests.get(url)
response = session.get(url, headers=headers)
response.raise_for_status()  # Проверка на ошибки
print(response.status_code)

# Парсинг HTML
tree = html.fromstring(response.content)

# Извлечение данных о фильмах
movies = tree.xpath('//table[@class="chart"]/tbody/tr')

# Подключение к MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['imdb_database']  # Название базы данных
collection = db['movies']  # Название коллекции

# Сохранение данных в MongoDB
for movie in movies:
    title = movie.xpath('.//td[@class="titleColumn"]/a/text()')
    year = movie.xpath('.//td[@class="titleColumn"]/span/text()')
    rating = movie.xpath('.//td[@class="ratingColumn imdbRating"]/strong/text()')
    
    if title and year and rating:  # Проверка на наличие данных
        movie_data = {
            'title': title[0],
            'year': year[0],
            'rating': rating[0]
        }
        print(movie_data)
        # Вставка в коллекцию
        collection.insert_one(movie_data)

print("Данные успешно сохранены в MongoDB.")


200
Данные успешно сохранены в MongoDB.


In [68]:
import requests
from lxml import html
from pymongo import MongoClient

def scrape_imdb_moviemeter():
    url = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3' # Замените на ваш User-Agent
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Проверка на ошибки HTTP
        tree = html.fromstring(response.content)
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к IMDb: {e}")
        return None

    movies = []
    for movie in tree.xpath('//div[contains(@class, "cli-title")]'):
        try:
            title = movie.xpath(".//div[contains(@class, 'cli-title')]/a/h3/text()")[0].strip()
            print(title)
            year = int(movie.xpath(".//div[contains(@class, 'cli-title-metadata')]/span/text()")[0].strip()[1:-1]) # Извлекаем год из скобок
            rating = float(movie.xpath(".//div[contains(@class, 'meter-const-ranking')]/text()")[0].strip())
            # ... здесь можно добавить извлечение других данных (например, режиссера, актеров) ...
            movies.append({
                "title": title,
                "year": year,
                "rating": rating
                # ... другие поля ...
            })
        except (IndexError, ValueError) as e:
            print(f"Ошибка при парсинге данных фильма: {e}") # Обработка ошибок при парсинге


    return movies


def save_to_mongodb(movies):
    client = MongoClient("mongodb://localhost:27017/") # Замените на ваши данные подключения
    db = client["imdb"] # имя базы данных
    collection = db["moviemeter"] # имя коллекции

    try:
        if movies:
            collection.insert_many(movies) # вставка нескольких документов
            print(f"Данные успешно сохранены в MongoDB.")
        else:
            print("Нет данных для сохранения.")
    except Exception as e:
        print(f"Ошибка при сохранении данных в MongoDB: {e}")
    finally:
        client.close()

if __name__ == "__main__":
    imdb_data = scrape_imdb_moviemeter()
    print(imdb_data)
    if imdb_data:
        save_to_mongodb(imdb_data)



Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of range
Ошибка при парсинге данных фильма: list index out of ran

In [ ]:
import requests
from lxml import html
from pymongo import MongoClient
import time
from fake_useragent import UserAgent
import time
from pprint import pprint
# Определение целевого URL
url = ""


ua = UserAgent()
# ua = {}
url = 'https://www.ebay.com'
headers = {'User-Agent': ua.safari}
# headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
session = requests.Session()
all_books = []
page = "/b/Fishing-Equipment-Supplies/1492/bn_1851047"
response = session.get(url + page, headers=headers)
print(response.status_code)
dom = html.fromstring(response.text)
print(dom)
items_list = []
items = dom.xpath('//li[contains(@class, "brwrvr__item-card brwrvr__item-card--list")]')
print(len(items))
for item in items:
    items_info = {}
    names = dom.xpath('.//h3[contains(@class, "textual-display")]')
    # print(names) # Выводит список текстов

    # # Извлекаем текст из каждого элемента
    name_info = [el.text_content().strip() for el in names]
    print(name_info)
    time.sleep(5)
    # links = dom.xpath('//h3[contains(@class, "textual-display")]/../@href')
    links = item.xpath('.//h3[@class="textual-display bsig__title__text"]/../@href')
    # print(len(links))
    # print(links)
    time.sleep(3)
    prises = item.xpath('.//span[contains(@class, "textual-display bsig__price bsig__price--displayprice")]')
    prises_info = [el.text_content().strip() for el in prises]
    # print(len(prises_info))
    time.sleep(3)
    info = item.xpath('.//span[contains(@class, "negative")]')
    # print(len(info))
    add_info = [el.text_content().strip() for el in info]
    # print(add_info)
    items_info['name_info'] = name_info
    items_info['links'] = links
    items_info['prises_info'] = prises_info
    items_info['add_info'] = add_info
    items_list.append(items_info)
pprint(items_list)


# //time/ancestor::*[contains(@class, '_topnews')]

200
<Element html at 0x1ee8d1a3660>
74
['Живые восковые черви, восковые черви рыбалка, кормушки для рептилий, бесплатная доставка', 'Живые гигантские личинки насекомых бесплатная доставка живой прибытия гарантия', 'Живые шипы, личинки синяя бутылка мухи личинки Бесплатная доставка, живая гарантия', 'Apple iPhone Pro 2-го поколения с чехлом для беспроводной зарядки MagSafe — США', 'Apple iPhone Pro 2-го поколения с чехлом для беспроводной зарядки MagSafe — (USB-C) — США', 'Светодиодный фонарик перчатки USB перезаряжаемый руки бесплатно удобный палец свет перчатки', 'Oros ® Strike индикатор 3-Pack * новый 2025 запас * BUNGS для рыбалки нахлыстом', 'Мощность вертлюги размер 5 8 10 удивительный прочность-кол X 50', 'Apple iPhone Pro 2-го поколения с чехлом для беспроводной зарядки MagSafe — (USB-C) — США', 'Суперзащитная маска на все лицо противотуманный экран безопасный прозрачный чехол на голову сделано в США', 'Рыболовная леска Zebco Omniflex Monofilament, 20 фунтов протестирована беспл

KeyboardInterrupt: 